In [37]:
import pandas as pd 
import requests 
import json

In [38]:
#api call and cleaning
bulk = requests.get('https://api.scryfall.com/bulk-data') #I learned that the uri for bulk data changes daily, so requesting the bulk data for the day then getting the uri for updated cards is needed (I had outdated data)
bulk_df = pd.json_normalize(bulk.json(),'data')
oracle_uri = bulk_df['download_uri'].loc[bulk_df.query('type == "oracle_cards"').index[0]]

response = requests.get(oracle_uri)
raw_oracle_cards = pd.json_normalize(response.json())

oracle_cards = raw_oracle_cards[[#'object', 'id', 
       'oracle_id', 
       #'multiverse_ids', 'mtgo_id',
       #'mtgo_foil_id', 'tcgplayer_id', 'cardmarket_id', 
       'name', 
       #'lang',
       #'released_at', 'uri', 'scryfall_uri', 
       'layout', 
       #'highres_image',
       #'image_status', 'mana_cost', 'cmc', 
       'type_line', 
       #'oracle_text',
       #'colors', 'color_identity', 'keywords', 'games', 'reserved',
       #'foil', 'nonfoil', 'finishes', 'oversized', 'promo', 'reprint',
       #'variation', 'set_id', 'set', 'set_name', 'set_type', 'set_uri',
       #'set_search_uri', 'scryfall_set_uri', 'rulings_uri',
       #'prints_search_uri', 'collector_number', 'digital', 
       #'rarity',
       #'flavor_text', 'card_back_id', 'artist', 'artist_ids',
       #'illustration_id', 'border_color', 'frame', 'full_art', 'textless',
       #'booster', 'story_spotlight', 'edhrec_rank', 'image_uris.small',
       #'image_uris.normal', 'image_uris.large', 'image_uris.png',
       #'image_uris.art_crop', 'image_uris.border_crop',
       'legalities.standard',
       #'legalities.future', 'legalities.historic',
       #'legalities.gladiator', 
       'legalities.pioneer',
       #'legalities.explorer', 
       'legalities.modern', 
       'legalities.legacy',
       'legalities.pauper', 
       'legalities.vintage', 
       #'legalities.penny',
       #'legalities.commander', 'legalities.brawl',
       #'legalities.historicbrawl', 'legalities.alchemy',
       #'legalities.paupercommander', 'legalities.duel',
       #'legalities.oldschool', 'legalities.premodern', 
       'prices.usd',
       #'prices.usd_foil', 'prices.usd_etched', 'prices.eur',
       #'prices.eur_foil', 
       'prices.tix', 
       #'related_uris.gatherer',
       #'related_uris.tcgplayer_infinite_articles',
       #'related_uris.tcgplayer_infinite_decks', 'related_uris.edhrec',
       #'security_stamp', 'preview.source', 'preview.source_uri',
       #'preview.previewed_at', 'power', 'toughness', 'penny_rank',
       #'arena_id', 'watermark', 'produced_mana', 'all_parts',
       'card_faces', 
       #'frame_effects', 'tcgplayer_etched_id',
       #'promo_types', 'loyalty', 'life_modifier', 'hand_modifier',
       #'attraction_lights', 'color_indicator', 'content_warning'
       ]]
oracle_cards = oracle_cards.rename(columns = {
    'name':'Name',
    'rarity':'Rarity',
    'legalities.pauper':'Pauper_Legal',
    'legalities.standard':'Standard_Legal',
    'legalities.pioneer':'Pioneer_Legal',
    'legalities.modern':'Modern_Legal',
    'legalities.legacy':'Legacy_Legal',
    'legalities.vintage':'Vintage_Legal',
    'prices.usd':'Price_USD',
    'prices.tix':'Price_Tix'
})

oracle_cards = oracle_cards.query("Standard_Legal == 'legal' or Pioneer_Legal == 'legal' or Modern_Legal == 'legal' or Legacy_Legal == 'legal' or Vintage_Legal == 'restricted' or Vintage_Legal == 'legal'")

card_faces = pd.json_normalize(oracle_cards['card_faces'].loc[~oracle_cards['card_faces'].isna()])
front_face = pd.json_normalize(card_faces[0])
back_face = pd.json_normalize(card_faces[1])

faces_index = 0 # we lose the relationship between the indices when we normalize the faces data, I'm not sure how to fix this with pandas but just keeping track of it like this works fine
for card_index in oracle_cards.loc[~oracle_cards['card_faces'].isna()].index:
    if oracle_cards['layout'].loc[card_index] == 'split':
        oracle_cards['Name'].loc[card_index] = front_face['name'].loc[faces_index]+'/'+back_face['name'].loc[faces_index]
    else:
        oracle_cards["Name"].loc[card_index] = front_face['name'].loc[faces_index]
    faces_index += 1


oracle_cards.head()

,oracle_id,Name,layout,type_line,Standard_Legal,Pioneer_Legal,Modern_Legal,Legacy_Legal,Pauper_Legal,Vintage_Legal,Price_USD,Price_Tix,card_faces
0,0004ebd0-dfd6-4276-b4a6-de0003e94237,Static Orb,normal,Artifact,not_legal,not_legal,not_legal,legal,not_legal,legal,21.46,0.51,NaN
1,0006faf6-7a61-426c-9034-579f2cfcfa83,Sensory Deprivation,normal,Enchantment — Aura,not_legal,legal,legal,legal,legal,legal,0.02,0.04,NaN
2,0007c283-5b7a-4c00-9ca1-b455c8dff8c3,Road of Return,normal,Sorcery,not_legal,not_legal,not_legal,legal,not_legal,legal,0.25,0.66,NaN
3,000d5588-5a4c-434e-988d-396632ade42c,Storm Crow,normal,Creature — Bird,not_legal,not_legal,legal,legal,legal,legal,0.21,0.03,NaN
4,000e5d65-96c3-498b-bd01-72b1a1991850,Walking Sponge,normal,Creature — Sponge,not_legal,not_legal,not_legal,legal,not_legal,legal,0.21,0.09,NaN


In [39]:
bulk = requests.get('https://api.scryfall.com/bulk-data') #I learned that the uri for bulk data changes daily, so requesting the bulk data for the day then getting the uri for updated cards is needed (I had outdated data)
bulk_df = pd.json_normalize(bulk.json(),'data')
default_uri = bulk_df['download_uri'].loc[bulk_df.query('type == "default_cards"').index[0]]

response = requests.get(default_uri)
default_cards = pd.json_normalize(response.json())


In [40]:
default_cards[['oracle_id','name','prices.usd']].head()

,oracle_id,name,prices.usd
0,44623693-51d6-49ad-8cd7-140505caf02f,Fury Sliver,0.38
1,8ae3562f-28b7-4462-96ed-be0cf7052ccc,Kor Outfitter,0.25
2,dc4e2134-f0c2-49aa-9ea3-ebf83af1445c,Spirit,0.10
3,9f0d82ae-38bf-45d8-8cda-982b6ead1d72,Siren Lookout,0.06
4,5aa12aff-db3c-4be5-822b-3afdf536b33e,Web,0.64


In [42]:
no_price_cards =  oracle_cards.loc[oracle_cards['Price_USD'].isna()] #dataframe for keeping track of the priceless cards

no_price_cards.head()

,oracle_id,Name,layout,type_line,Standard_Legal,Pioneer_Legal,Modern_Legal,Legacy_Legal,Pauper_Legal,Vintage_Legal,Price_USD,Price_Tix,card_faces
7,00185f3b-2777-4417-a8e0-4691f41c0ec1,Wyluli Wolf,normal,Creature — Wolf,not_legal,not_legal,not_legal,legal,legal,legal,None,0.04,NaN
13,0023888e-7bec-43e0-8dee-d1a4eb94b372,Palinchron,normal,Creature — Illusion,not_legal,not_legal,not_legal,legal,not_legal,legal,None,0.03,NaN
17,002ad5fa-a20c-4d30-beef-401d89868d67,Wei Strike Force,normal,Creature — Human Soldier,not_legal,not_legal,not_legal,legal,legal,legal,None,0.06,NaN
29,004584f8-fd89-4a3e-9782-c117dbe1532b,Weakstone,normal,Artifact,not_legal,not_legal,not_legal,legal,not_legal,legal,None,0.05,NaN
55,008011e2-7b82-4962-af6e-be627112f37f,Theft of Dreams,normal,Sorcery,not_legal,not_legal,not_legal,legal,legal,legal,None,0.03,NaN


In [56]:
for oraclecard_index in no_price_cards.index:#for each card we don't have price for in the oracle card list
    card_printings = default_cards.query('oracle_id == "'+no_price_cards['oracle_id'].loc[oraclecard_index]+'"') #make a temporary dataframe for each printing that occurs in the defaulat card list
    prices_list = []
    for printing_index in card_printings.index: # for each printing of the card
        if card_printings['prices.usd'].loc[printing_index]: #if we can find a nonzero price, 
            prices_list.append(card_printings['prices.usd'].loc[printing_index]) #add it to the list of prices
    #make the price the lowest found!
    if prices_list != []:
        no_price_cards['Price_USD'].loc[oraclecard_index] = min(prices_list)


no_price_cards['Price_USD'].head()

C:\Users\jackb\AppData\Local\Temp\ipykernel_9948\3164998165.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_price_cards['Price_USD'].loc[oraclecard_index] = min(prices_list)


7      0.67
13    36.07
17     3.46
29    15.45
55     0.21
Name: Price_USD, dtype: object

# FFFFUUUUUCCCKKKKIIINNNGGGGGG FOIL ONLY CARDS!!!!!!!!

In [61]:
no_price_cards.loc[no_price_cards['Price_USD'].isna()]


,oracle_id,Name,layout,type_line,Standard_Legal,Pioneer_Legal,Modern_Legal,Legacy_Legal,Pauper_Legal,Vintage_Legal,Price_USD,Price_Tix,card_faces
547,04aa210a-235f-4e07-87d1-0d28cdf6888b,"Huatli, Dinosaur Knight",normal,Legendary Planeswalker — Huatli,not_legal,legal,legal,legal,not_legal,legal,None,0.02,NaN
1236,0b64da77-ca1c-427b-b48b-b919f0fe08e9,"Glenn, the Voice of Calm",normal,Legendary Creature — Human Advisor,not_legal,not_legal,not_legal,legal,not_legal,legal,None,None,NaN
1531,0e084342-31ff-47a5-9fd8-5b385add7a1a,"Saheeli, the Gifted",normal,Legendary Planeswalker — Saheeli,not_legal,not_legal,not_legal,legal,not_legal,legal,None,None,NaN
2348,15474e82-94b4-47e1-b91c-3deaf4dbffd6,"Ajani, Inspiring Leader",normal,Legendary Planeswalker — Ajani,not_legal,legal,legal,legal,not_legal,legal,None,0.14,NaN
2434,1601e553-31c2-4e04-9f0c-bc17e49eeff7,"Basri, Devoted Paladin",normal,Legendary Planeswalker — Basri,not_legal,legal,legal,legal,not_legal,legal,None,0.02,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26801,f3ad60a6-05d0-48a1-a804-7900d306d613,"Liliana, the Necromancer",normal,Legendary Planeswalker — Liliana,not_legal,legal,legal,legal,not_legal,legal,None,None,NaN
26895,f4905c40-003e-4992-b8d7-3f07ba09c686,"Hanweir, the Writhing Township",meld,Legendary Creature — Eldrazi Ooze,not_legal,legal,legal,legal,not_legal,legal,None,None,NaN
27085,f6479f7e-01f4-49f1-a444-04bf38934f6b,Marneus Calgar,normal,Legendary Creature — Astartes Warrior,not_legal,not_legal,not_legal,legal,not_legal,legal,None,None,NaN
27792,fc762e40-84c1-4762-82f7-a0fc4f6fd522,"Garruk, Savage Herald",normal,Legendary Planeswalker — Garruk,not_legal,legal,legal,legal,not_legal,legal,None,0.36,NaN
